## 1) 데이터 준비와 전처리
--- 
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


* Pandas를 사용하여 movielens 데이터를 불러옵니다. 

In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


* 1,000,209개의 영화 데이터 중, rating 이 3점 이상인 영화는 83.63%로 836,478개 입니다. 

In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.



## 2) 분석해 봅시다.
---
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [5]:
# 영화 개수
print ('영화 개수 : ', ratings['movie_id'].nunique())

영화 개수 :  3628


In [6]:
# 사용자 수
ratings['user_id'].nunique()

6039

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings = ratings.join(movies.set_index('movie_id'), on = 'movie_id')
ratings.head(10)

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
5,1,1197,3,978302268,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,978302039,Ben-Hur (1959),Action|Adventure|Drama
7,1,2804,5,978300719,"Christmas Story, A (1983)",Comedy|Drama
8,1,594,4,978302268,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
9,1,919,4,978301368,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


* movie_id 로만 어떤 영화인지 파악할 수 없기 때문에 movies 데이터를 movie_id 에 맞춰서 ratings 데이터에 붙여줍니다.

In [9]:
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head(10)

,user_id,movie_id,counts,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
5,1,1197,3,"Princess Bride, The (1987)"
6,1,1287,5,Ben-Hur (1959)
7,1,2804,5,"Christmas Story, A (1983)"
8,1,594,4,Snow White and the Seven Dwarfs (1937)
9,1,919,4,"Wizard of Oz, The (1939)"


* 사용하지 않을 'timestamp'와 'genre'는 데이터에서 삭제 해줍니다. 

In [10]:
movie_count = ratings.groupby("movie_id")["user_id"].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [11]:
movie_count = ratings.groupby("title")["user_id"].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
user_count = ratings.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
---

In [13]:
# 내가 좋아하는 영화 데이터 추가. 
my_favorite_movieid = [1 , 2 , 2000 ,1835 ,3825]
my_favorite_movietitle = []

for i in my_favorite_movieid:
    my_favorite_movietitle.extend(list(movies[movies['movie_id'] == i]['title']))
    
# 'hyumin'이라는 user_id가 위 영화 rating을 전부 5로 줬다고 가정합니다.
my_movielist = pd.DataFrame({'user_id': ['hyumin']*5, 'movie_id': my_favorite_movieid, 'counts':[5]*5, 'title':my_favorite_movietitle})

if not ratings.isin({'user_id':['hyumin']})['user_id'].any():  # user_id에 'hyumin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title
1000203,6040,1090,3,Platoon (1986)
1000205,6040,1094,5,"Crying Game, The (1992)"
1000206,6040,562,5,Welcome to the Dollhouse (1995)
1000207,6040,1096,4,Sophie's Choice (1982)
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982)
0,hyumin,1,5,Toy Story (1995)
1,hyumin,2,5,Jumanji (1995)
2,hyumin,2000,5,Lethal Weapon (1987)
3,hyumin,1835,5,City of Angels (1998)
4,hyumin,3825,5,Coyote Ugly (2000)


* 우리가 만든 추천 시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 
* 아래와 같이 내가 좋아하는 영화들을 기존의 데이터에 추가해서 넣습니다.

## 4) CSR matrix를 직접 만들어 봅시다.
---

In [14]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['hyumin'])    
print(movie_to_idx['Coyote Ugly (2000)'])

6039
2047


In [21]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings): 
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing Fail!!
title column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
0,6039,1,5,40
1,6039,2,5,513
2,6039,2000,5,648
3,6039,1835,5,1418


* user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구합니다. 
* movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱 합니다.

In [23]:
# CSR matrix 생성

from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

---

In [24]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.

* factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
* regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
* use_gpu : GPU를 사용할 것인지
* iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [36]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=1000, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [37]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [38]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델 학습이 끝났습니다! 이제 2가지를 아래 사항을 살펴보도록 하겠습니다.

1. Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
---

In [39]:
hyumin, Coyote_Ugly = user_to_idx['hyumin'], movie_to_idx['Coyote Ugly (2000)']
hyumin_vector, Coyote_Ugly_vector = als_model.user_factors[hyumin], als_model.item_factors[Coyote_Ugly]

print('슝=3')

슝=3


In [40]:
hyumin_vector

array([ 3.03173184e-01, -3.30197155e-01, -1.01762027e-01,  1.77621573e-01,
       -1.04570955e-01,  1.95667863e-01, -1.19425379e-01,  2.84207582e-01,
       -2.99052875e-02,  2.54514009e-01, -1.61020495e-02, -4.72999401e-02,
        6.38182536e-02,  8.52714032e-02,  1.12186588e-01, -1.26975387e-01,
       -2.60360390e-01,  3.00036967e-01, -4.24469471e-01, -5.39410003e-02,
       -1.33796811e-01, -1.49703592e-01,  6.31414279e-02,  1.82803541e-01,
       -1.77452698e-01, -4.38565969e-01, -2.55355477e-01,  3.80034119e-01,
       -6.01363301e-01,  2.82397747e-01,  1.65698051e-01,  1.01414561e-01,
        2.11841404e-01,  4.88893948e-02,  4.48081583e-01,  6.62591308e-03,
        1.02159142e-01, -7.38087177e-01, -2.63727129e-01, -1.47417143e-01,
       -1.51306406e-01, -2.52131969e-01,  2.30864510e-01,  9.75394249e-02,
       -1.84356272e-01, -9.71260816e-02, -5.84647000e-01,  5.48939645e-01,
       -3.35620672e-01, -3.81632745e-01,  2.35345557e-01,  3.42338443e-01,
        1.16065904e-01, -

In [41]:
Coyote_Ugly_vector

array([ 6.73265895e-03,  1.39110978e-03,  1.05717918e-02,  6.15274487e-03,
       -1.68711715e-03,  1.16684018e-02,  7.35049369e-03,  5.98787796e-03,
       -3.25387181e-03,  1.65552013e-02,  1.18552931e-02,  6.49142498e-03,
        1.28403334e-02,  7.54570961e-03,  6.27795700e-03,  4.12284536e-03,
       -4.57381085e-03, -8.31153244e-04,  4.58156166e-04,  5.24470396e-03,
        3.56251863e-03,  8.64002388e-04,  9.18990094e-03,  1.11369416e-02,
       -1.96173089e-03,  1.84813340e-03, -7.28618645e-04,  9.74126440e-03,
        4.72709071e-03,  7.67918536e-03,  4.09274828e-03,  3.83358728e-03,
        3.21433484e-03, -2.82045366e-05,  1.36055108e-02,  3.08863795e-03,
        6.45615364e-05,  2.52134097e-03,  5.59120392e-03,  2.23175809e-03,
        1.38546852e-03,  6.34571479e-04,  9.51707829e-03,  6.26725750e-03,
        1.08589642e-02,  5.42392768e-03, -4.95826453e-03,  6.60262583e-03,
        2.56420649e-03,  7.38875475e-03,  2.46507511e-03,  5.82064269e-03,
        8.50700028e-03, -

In [42]:
# hyumin과 Coyote_Ugly_vector를 내적하는 코드
np.dot(hyumin_vector, Coyote_Ugly_vector)

0.66732436

In [45]:
american_beauty = movie_to_idx['American Beauty (1999)']
american_beauty_vector = als_model.item_factors[american_beauty]
np.dot(hyumin_vector, american_beauty_vector)

0.013047494

* 좋아하는 영화 리스트에 없었던 'American Beauty (1999)'의 수치가 리스트에 있었던 'Coyote Ugly (2000)'보다 훨씬 낮은 수치를 보여주고 있습니다. 

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
---

In [46]:
favorite_movie = 'Toy Story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 0.9999999),
 (3589, 0.47162613),
 (3533, 0.46755034),
 (3494, 0.46725863),
 (3617, 0.46722808),
 (3495, 0.46701267),
 (3606, 0.46659806),
 (3624, 0.46646106),
 (3625, 0.46635637),
 (3587, 0.46625704),
 (3510, 0.46624494),
 (3288, 0.46622345),
 (3381, 0.4661898),
 (3563, 0.46611068),
 (3598, 0.46610278)]

In [47]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Toy Story (1995)',
 'Soft Toilet Seats (1999)',
 'To Have, or Not (1995)',
 'An Unforgettable Summer (1994)',
 'Slappy and the Stinkers (1998)',
 'Circus (2000)',
 'Truce, The (1996)',
 'Slaughterhouse (1987)',
 'Promise, The (Versprechen, Das) (1994)',
 'Shopping (1994)',
 'Bittersweet Motel (2000)',
 'Hatchet For the Honeymoon (Rosso Segno Della Follia) (1969)',
 'Mascara (1999)',
 'Better Living (1998)',
 'Talk of Angels (1998)']

In [48]:
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id, N=15)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [49]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Soft Toilet Seats (1999)',
 'To Have, or Not (1995)',
 'An Unforgettable Summer (1994)',
 'Slappy and the Stinkers (1998)',
 'Circus (2000)',
 'Truce, The (1996)',
 'Slaughterhouse (1987)',
 'Promise, The (Versprechen, Das) (1994)',
 'Shopping (1994)',
 'Bittersweet Motel (2000)',
 'Hatchet For the Honeymoon (Rosso Segno Della Follia) (1969)',
 'Mascara (1999)',
 'Better Living (1998)',
 'Talk of Angels (1998)']

In [50]:
get_similar_movie('Jumanji (1995)')

['Jumanji (1995)',
 'Dangerous Game (1993)',
 'Zachariah (1971)',
 'Five Wives, Three Secretaries and Me (1998)',
 'Slappy and the Stinkers (1998)',
 'Railroaded! (1947)',
 'Yankee Zulu (1994)',
 'Napoleon and Samantha (1972)',
 'Second Jungle Book: Mowgli & Baloo, The (1997)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'Alley Cats, The (1968)',
 'Daens (1992)',
 "Another Man's Poison (1952)",
 'Theodore Rex (1995)',
 'Last Klezmer: Leopold Kozlowski, His Life and Music, The (1995)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
---

In [51]:
user = user_to_idx['hyumin']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(901, 0.12513724),
 (2294, 0.116403535),
 (1671, 0.115882844),
 (1039, 0.1070852),
 (1350, 0.10372733),
 (1414, 0.102476105),
 (792, 0.09929678),
 (418, 0.09890476),
 (1496, 0.09824666),
 (650, 0.09630306),
 (770, 0.09549129),
 (2603, 0.094955236),
 (1234, 0.09276271),
 (2413, 0.09239371),
 (427, 0.08941677),
 (615, 0.08713067),
 (486, 0.086784035),
 (1982, 0.085817754),
 (136, 0.085784554),
 (367, 0.08513732)]

In [52]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Center Stage (2000)',
 'Autumn in New York (2000)',
 'Skulls, The (2000)',
 'Fallen (1998)',
 'Mansfield Park (1999)',
 'Mildred Pierce (1945)',
 'What Lies Beneath (2000)',
 'What Dreams May Come (1998)',
 'Peacemaker, The (1997)',
 'Lethal Weapon 2 (1989)',
 'Cliffhanger (1993)',
 'Here on Earth (2000)',
 'Dave (1993)',
 'Duets (2000)',
 'Interview with the Vampire (1994)',
 'Cocoon (1985)',
 'Remember the Titans (2000)',
 'Space Jam (1996)',
 'Desperado (1995)',
 'Grease (1978)']

In [64]:
center_stage = movie_to_idx['Center Stage (2000)']
explain = als_model.explain(user, csr_data, itemid=center_stage)

In [65]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Coyote Ugly (2000)', 0.11670640831648568),
 ('City of Angels (1998)', 0.017563488839626537),
 ('Lethal Weapon (1987)', 0.015356136874840762),
 ('Toy Story (1995)', -0.006624606374916025),
 ('Jumanji (1995)', -0.016829316845973847)]

* Center Stage (2000)을 추천 해주는데 있어서 Coyote Ugly (2000)이 0.1의 수로 기여한 것으로 보입니다.
* 언듯 보았을 때, 그닥 높은 기여도로 보이진 않습니다.

* Netflix, Youtube, 티빙, 쿠팡플레이 등 여러 회사들이 사용자가 좋아하는 영상으로 추천시스템을 사용하여 좋아할만한 영상을 추천해주고 있다. 내가 좋아하는 영상을 5개 픽 했음에도 불구하고, 정확히 얼마나 영향을 미치고 있는지 그리고 정말 내가 추천해준 영화들을 좋아하는지는 굉장히 주관적이라고 생각된다. 
* AI 추천 모델임에도 불구하고 주관적인 요점이 영향을 미치는게 굉장히 매력적인 요소이고, 더욱 더 공부가 필요한 분야라고 생각된다.